_Learning to build a simple data warehouse gives me practical insight into how data flows from raw sources to actionable insights. Even though full-scale architecture is the responsibility of data architects, understanding the basics of modelling, ETL, and schema design helps me collaborate better with engineers, solve data issues more effectively, and approach analytics with a true end-to-end perspective._

<br>

### **Step 1** | Scoping out the project

---

#### Objective

Develop a modern data warehouse using SQL Server to consolidate sales data, enabling analytical reporting and informed decision-making.

#### Specifications

- **Data Sources**: Import data from two source systems (ERP and CRM) provided as CSV files.
- **Data Quality**: Cleanse and resolve data quality issues prior to analysis.
- **Integration**: Combine both sources into a single, user-friendly data model designed for analytical queries.
- **Scope**: Focus on the latest dataset only; historization of data is not required.
- **Documentation**: Provide clear documentation of the data model to support both business stakeholders and analytics teams.

#### Medallion Architecture

As per the course’s recommendation, we will be using the Medallion data management paradigm. This layered approach is particularly effective for building a modern data warehouse because it separates raw, cleansed, and curated datasets into clear stages—commonly referred to as Bronze, Silver, and Gold layers. By structuring the pipeline this way, we ensure that each layer has a distinct purpose: Bronze holds unaltered, raw source data; Silver refines and cleanses this data for reliability; and Gold aggregates it into business-ready tables optimised for reporting and analytics.

Opting for Medallion provides several advantages. It improves data quality and trust by ensuring transformations are traceable and reproducible, as all data first lands in a raw state before being standardised and validated. It also enhances maintainability and scalability, allowing us to debug issues at the appropriate layer rather than across the entire warehouse. 

<br>

![data architecture figure 1 image](assets/img/journal_fig1.png)

<br>


| | Bronze Layer | Silver Layer | Gold Layer |
| - | - | - | - |
| **Definition** | Raw, unprocessed data as-is from sources | Clean and standardised data | Business-ready data | 
| **Objective** | Traceability & debugging | (Intermediate layer) Prepare data for analysis | Provide data to be consumed for reporting & analytics |
| **Object Type** | Tables | Tables | Views |
| **Load Method** | Full load (truncate & insert) | Full load (truncate & insert) | None |
| **Data Transformation** | None (as-is) | Data cleaning, standardisation, normalisation, enrichment & derived columns | Data integration, aggregation, business logic & rules |
| **Data Modeling** | None (as-is) | None (as-is) | Start schema, aggregated objects, flat tables |
| **Target Audience** | Data engineers | Data engineers & analysts | Data analysts & business users |  

<br>

#### Layers for Separation of Concerns (SoC)

The above layers mean that we have separation of concerns (SoC) - an important principle where we take a complex system and break it down into independent parts, each focused on a specific responsibility or operation without overlapping with others. So for a data warehouse, SoC means breaking the architecture into independent layers—such as ingestion, transformation, storage, and consumption—so each layer handles its own responsibility without interfering with the others...

<br>

![data architecture figure 2 image](assets/img/journal_fig2.png)

#### Beginnings...

Moving forward, this journal will document each stage of the data warehouse build with clear, structured notes. We will capture:

1. **Data Source Overview** – Key details about each source, including its format, refresh frequency, volume, and method of access.
2. **Data Quality and Validation** – How we identify and resolve issues such as missing data, duplicates, and inconsistencies, as well as checks between Bronze and Silver layers.
3. **Medallion Layer Objectives** – The purpose and responsibilities of the Bronze, Silver, and Gold layers, along with how each supports data trust and reporting readiness.
4. **Target Schema Design** – Sketches and notes on fact and dimension tables, including any decisions around star vs. snowflake schema design.
5. **ETL / ELT Flow** – Steps for moving data between layers, whether through incremental or full loads, and how the process will be orchestrated.
6. **Testing and Monitoring** – Plans for data validation, quality checks, and ongoing monitoring to ensure the reliability of the Gold outputs.
7. **Versioning and Progress Log** – A running journal of decisions, challenges, lessons learned, and key milestones as the project evolves.

#### The high-level goal

![data architecture figure 3 image](assets/img/journal_fig3.png)


### **Step 2** | Establishing rules & naming conventions

---

#### Naming Conventions

We need naming conventions to keep our data systems clear, consistent, and scalable. They let us understand datasets and pipelines at a glance, reduce errors, and make automation with tools like Airflow or dbt much easier. Consistent names help us collaborate better, onboard new team members faster, and simplify governance tasks like lineage and documentation. With predictable patterns in place, our Medallion or multi-source warehouse can grow without becoming messy or confusing.

#### General Principles
- **Naming Conventions**: Use snake_case, with lowercase letters and underscores (_) to separate words.
- **Language**: Use English for all names.
- **Avoid Reserved Words**: Do not use SQL reserved words as object names.

#### Table Naming Conventions

**Bronze Layer Rules**

All names must start with the source system name, and table names must match their original names without renaming.
- `[sourcesystem_entity]`
    - `[sourcesystem]`: Name of the source system (e.g., crm, erp).
    - `[entity]`: Exact table name from the source system.
    - Example: `crm_customer_info` → Customer information from the CRM system.

**Silver Layer Rules**

In this scenario, we are not renaming items between Bronze and Silver. So the rules above will apply in Silver. 

**Gold Layer Rules**

All names must use meaningful, business-aligned names for tables, starting with the category prefix.
- `[category_entity]`
    - `[category]`: Describes the role of the table, such as dim (dimension) or fact (fact table).
    - `[entity]`: Descriptive name of the table, aligned with the business domain (e.g., customers, products, sales).
    - Examples:
        - `dim_customers` → Dimension table for customer data.
        - `fact_sales` → Fact table containing sales transactions.

**Glossary of Category Patterns**
| Pattern | Meaning | Example(s) |
| - | - | - |
| `dim_` | Dimension table | `dim_customer`, `dim_product` | 
| `fact_` | Fact table | `fact_sales` | 
| `agg_` | Aggregated table | `agg_customer`, `agg_sales_monthly` | 


#### Column Naming Conventions

**Surrogate Keys**

- All primary keys in dimension tables must use the suffix `_key`.
- `[table_name]_key`
    - [table_name]: Refers to the name of the table or entity the key belongs to.
    - `_key`: A suffix indicating that this column is a surrogate key.
    - Example: `customer_key` → Surrogate key in the dim_customers table.

**Technical Columns**

- All technical columns must start with the prefix dwh_, followed by a descriptive name indicating the column's purpose.
- `dwh_[column_name]`
    - `dwh`: Prefix exclusively for system-generated metadata.
    - `[column_name]`: Descriptive name indicating the column's purpose.
    - Example: `dwh_load_date` → System-generated column used to store the date when the record was loaded.

**Stored Procedure**

All stored procedures used for loading data must follow the naming pattern: 
- `load_[layer]`.
    - `[layer]`: Represents the layer being loaded, such as bronze, silver, or gold.
    - Example:
        - `load_bronze` → Stored procedure for loading data into the Bronze layer.
        - `load_silver` → Stored procedure for loading data into the Silver layer.


### **Step 3** | Ready the database & schema

---

To begin capturing our design into code - we need to create and select a database, followed by establishing the schemas (which reflect our layers).

A key rule to note with any shared scripts - particularly high-level ones (see below), is to include a clear description for each, as well as warnings regarding its purpose. 

In [ ]:
/*
=============================================================
Create Database and Schemas
=============================================================
Script Purpose:
    This script creates a new database named 'DataWarehouse' after checking if it already exists. 
    If the database exists, it is dropped and recreated. Additionally, the script sets up three schemas 
    within the database: 'bronze', 'silver', and 'gold'.
	
W A R N I N G :
    Running this script will drop the entire 'DataWarehouse' database if it exists. 
    All data in the database will be permanently deleted. Proceed with caution 
    and ensure you have proper backups before running this script.
*/

USE master;
GO

-- Drop and recreate the 'DataWarehouse' database
IF EXISTS (SELECT 1 FROM sys.databases WHERE name = 'DataWarehouse')
BEGIN
    ALTER DATABASE DataWarehouse SET SINGLE_USER WITH ROLLBACK IMMEDIATE;
    DROP DATABASE DataWarehouse;
END;
GO

-- create the DWH database
CREATE DATABASE DataWarehouse;
GO

USE DataWarehouse;
GO

-- create schemas 
CREATE SCHEMA bronze;
GO

CREATE SCHEMA silver;
GO

CREATE SCHEMA gold;
GO


### **Step 4** | Developing the Bronze Layer

---

For us to begin building the first layer of our warehouse's architecture, we like any discipline, need to carefully **understand the sources** and the context surrounding them. We need to meet with stakeholders, and gauge:

**Business Context & Ownership**
- Who owns the data?
- What business process it supports?
- Systems and data documentation
- Data model and data catalog 

**Architecture & Technology Stack**
- How is data stored? 
- What are the integration capabilities?

**Extract & Load**
- Incremental vs. full load?
- Data scope & historical needs
- What is the expected size of the extracts?
- Are there any data volume limitations?
- How to avoid impacting the source system's performance?
- Authentication and authorisation 

#### Specification of the Bronze Layer

The Bronze Layer will handle raw, un-processed data as-is from sources. The overall objective of this layer is traceability and debugging.

We are doing a full-load (truncate and insert), producing tables.

To get started, we will explore the data to identify the column names and data types (ie. **Data Profiling**).

_previewing 8 of 15000+ lines_

Referencing the data source above, and the naming conventions we set for each layer, we create our DDL (Data Definition Language). We repeat this process for each table to a total of six. These will store the raw data from the two sources folders (each holding three CSV files respectively).

In [ ]:
-- if this table exists, remove it and create a new one
IF OBJECT_ID ('bronze.crm_prd_info', 'U')  IS NOT NULL
    DROP TABLE bronze.crm_prd_info;
-- table creation with the required columns
CREATE TABLE bronze.crm_prd_info (
    prd_id INT,
    prd_key NVARCHAR(50),
    prd_nm NVARCHAR(50),
    prd_cost INT,
    prd_line NVARCHAR(2),
    prd_start_dt DATE,
    prd_end_dt DATE
);

Once complete, we have six tables in the Bronze layer schema, of which all clearly reference their source system. 

```
└── Bronze Layer/                        # schema           
    ├── bronze.crm_cust_info/            # table for CRM 
    ├── bronze.crm_prd_info/
    ├── bronze.crm_sales_details/        
    ├── bronze.erp_cust_az12/            # table for ERP   
    ├── bronze.erp_loc_a101/        
    └── bronze.erp_px_cat_gtv2/
```

Following the structure setup, we then load the data via a bulk, truncate and insert process. 

In [ ]:
-- Prep the table for first load by making sure it is empty, or if previously loaded, avoid a duplication error
TRUNCATE TABLE bronze.crm_cust_info;

-- load the data from file
BULK INSERT bronze.crm_cust_info
FROM 'C:\Users\Bagheera\My Drive\07 DataEng\sql-data-warehouse-project\datasets\source_crm\cust_info.csv'
-- below is we where provide the specification for the upload
WITH (
    FIRSTROW = 2, -- skip header row
    FIELDTERMINATOR = ',', -- specify the field delimiter
    TABLOCK
);

-- check results
SELECT * FROM bronze.crm_cust_info;

-- check row count exc. the first row
SELECT COUNT(*) FROM bronze.crm_cust_info;

Once we have basic structure in place, we can repeat for each data source, adding extensive formatting, keywords including `PRINT`, `TRY`, `DECLARE` throughout the code as a stored procedure. Doing such provides users with critical feedback, particularly in both system status and maintainance/debugging. This heavily increases the lines of code, but as Baraa Salkini notes:
> _'...this is what data engineering is all about, not just loading the data, but engineering the entire pipeline, measuring the speed, accounting for errors, to print and capture each step in the ETL process.'_

In [ ]:
CREATE OR ALTER PROCEDURE dbo.load_bronze AS 
BEGIN
    DECLARE @start_time DATETIME, @end_time DATETIME, @batch_start_time DATETIME, @batch_end_time DATETIME;
    BEGIN TRY
        SET @batch_start_time = GETDATE();
        PRINT '====================================';
        PRINT 'Loading data into bronze layer...';
        PRINT '====================================';
        PRINT '------------------------------------';
        PRINT 'Loading CRM tables ...';
        PRINT '------------------------------------';
        SET @start_time = GETDATE()
        PRINT '>> Truncating table: bronze.crm_cust_info';
        -- Prep the table for first load by making sure it is empty, or if previously loaded, avoid a duplication error
        TRUNCATE TABLE bronze.crm_cust_info;
        -- load the data from file
        BULK INSERT bronze.crm_cust_info
        FROM 'C:\Users\Bagheera\My Drive\07 DataEng\sql-data-warehouse-project\datasets\source_crm\cust_info.csv'
        -- below is we where provide the specification for the upload
        WITH (
            FIRSTROW = 2, -- skip header row
            FIELDTERMINATOR = ',', -- specify the field delimiter
            TABLOCK
        );
        SET @end_time = GETDATE()
        PRINT '>> Load Duration: ' + CAST(DATEDIFF(second, @start_time, @end_time) AS NVARCHAR(10)) + ' seconds';

        SET @start_time = GETDATE()
        PRINT '>> Truncating table: bronze.crm_prd_info';
        TRUNCATE TABLE bronze.crm_prd_info;
        BULK INSERT bronze.crm_prd_info
        FROM 'C:\Users\Bagheera\My Drive\07 DataEng\sql-data-warehouse-project\datasets\source_crm\prd_info.csv'
        WITH (
            FIRSTROW = 2, 
            FIELDTERMINATOR = ',', 
            TABLOCK
        );
        SET @end_time = GETDATE()
        PRINT '>> Load Duration: ' + CAST(DATEDIFF(second, @start_time, @end_time) AS NVARCHAR(10)) + ' seconds';

        SET @start_time = GETDATE()
        PRINT '>> Truncating table: bronze.sales_details';
        TRUNCATE TABLE bronze.crm_sales_details;
        BULK INSERT bronze.crm_sales_details
        FROM 'C:\Users\Bagheera\My Drive\07 DataEng\sql-data-warehouse-project\datasets\source_crm\sales_details.csv'
        WITH (
            FIRSTROW = 2,
            FIELDTERMINATOR = ',', 
            TABLOCK
        );
        SET @end_time = GETDATE()
        PRINT '>> Load Duration: ' + CAST(DATEDIFF(second, @start_time, @end_time) AS NVARCHAR(10)) + ' seconds';

        PRINT '------------------------------------';
        PRINT 'Loading ERP tables ...';
        PRINT '------------------------------------';
        SET @start_time = GETDATE()
        PRINT '>> Truncating table: bronze.cust_az12';
        TRUNCATE TABLE bronze.erp_cust_az12;
        BULK INSERT bronze.erp_cust_az12
        FROM 'C:\Users\Bagheera\My Drive\07 DataEng\sql-data-warehouse-project\datasets\source_erp\cust_az12.csv'
        WITH (
            FIRSTROW = 2, 
            FIELDTERMINATOR = ',', 
            TABLOCK
        );
        SET @end_time = GETDATE()
        PRINT '>> Load Duration: ' + CAST(DATEDIFF(second, @start_time, @end_time) AS NVARCHAR(10)) + ' seconds';
        SET @start_time = GETDATE()
        PRINT '>> Truncating table: bronze.loc_101';
        TRUNCATE TABLE bronze.erp_loc_a101;
        BULK INSERT bronze.erp_loc_a101
        FROM 'C:\Users\Bagheera\My Drive\07 DataEng\sql-data-warehouse-project\datasets\source_erp\loc_a101.csv'
        WITH (
            FIRSTROW = 2, 
            FIELDTERMINATOR = ',', 
            TABLOCK
        );
        SET @end_time = GETDATE()
        PRINT '>> Load Duration: ' + CAST(DATEDIFF(second, @start_time, @end_time) AS NVARCHAR(10)) + ' seconds';
        SET @start_time = GETDATE()
        PRINT '>> Truncating table: px_cat_g1v2';
        TRUNCATE TABLE bronze.erp_px_cat_g1v2;
        BULK INSERT bronze.erp_px_cat_g1v2
        FROM 'C:\Users\Bagheera\My Drive\07 DataEng\sql-data-warehouse-project\datasets\source_erp\px_cat_g1v2.csv'
        WITH (
            FIRSTROW = 2,
            FIELDTERMINATOR = ',',
            TABLOCK
        );
        SET @end_time = GETDATE()
        PRINT '>> Load Duration: ' + CAST(DATEDIFF(second, @start_time, @end_time) AS NVARCHAR(10)) + ' seconds';
        
        SET @batch_end_time = GETDATE();
        PRINT '====================================';
        PRINT 'Data loaded into bronze layer successfully.';
        PRINT 'Total Load Duration: ' + CAST(DATEDIFF(second, @batch_start_time, @batch_end_time) AS NVARCHAR(10)) + ' seconds';
        PRINT '====================================';
    END TRY
    BEGIN CATCH
        PRINT '====================================';
        PRINT 'Error occurred while loading data into Bronze layer:';
        PRINT 'Error Message ' + ERROR_MESSAGE();
        PRINT 'Error Message ' + CAST(ERROR_NUMBER() AS NVARCHAR(10));     
        PRINT 'Error Message ' + CAST(ERROR_STATE() AS NVARCHAR(10));  
        PRINT '====================================';
    END CATCH
END


So that `EXEC dbo.load_bronze;` returns...

```
Started executing query at Line 1
====================================
Loading data into bronze layer...
====================================
------------------------------------
Loading CRM tables ...
------------------------------------
>> Truncating table: bronze.crm_cust_info
(18493 rows affected)
>> Load Duration: 0 seconds
>> Truncating table: bronze.crm_prd_info
(397 rows affected)
>> Load Duration: 0 seconds
>> Truncating table: bronze.sales_details
(60398 rows affected)
>> Load Duration: 1 seconds
------------------------------------
Loading ERP tables ...
------------------------------------
>> Truncating table: bronze.cust_az12
(18483 rows affected)
>> Load Duration: 0 seconds
>> Truncating table: bronze.loc_101
(18484 rows affected)
>> Load Duration: 0 seconds
>> Truncating table: px_cat_g1v2
(37 rows affected)
>> Load Duration: 0 seconds
====================================
Data loaded into bronze layer successfully.
Total Load Duration: 1 seconds
====================================
Total execution time: 00:00:00.294
``` 

### **Step 5** | Developing the Silver Layer

---

#### Capturing the Data Flow

With our source data _entering the warehouse_, we need to keep track of where it is going. We create a data flow diagram to capture where the data comes from, where it ends up, and how it moves through each layer. While similar to a high-level architecture diagram, it focuses solely on the data itself. This helps us develop the data lineage—a critical concept for maintaining reliable data pipelines.

<br>

![data architecture figure 4 image](assets/img/journal_fig4.png)

<br>

#### Workflow for Silver Layer

Similar to Bronze, where we needed to explore the context and requirements. With Silver we need to explore and understand the data itself, principally because we are transitioning from raw data, to clean, standardiised data. In order to achieve this, we follow these steps.

1. Review the data in its Bronze layer state.
2. Data cleaning
    - Check data quality
    - Write data transformations
    - Insert into silver layer
3. Perform data correctness checks
    - If errors or concerns appear -> cycle back to previous steps
4. Data documentation

#### Reviewing the data & anticipating an integration model

To quickly preview a table in _SQL Server_, we can right click on table in the connections directory and 'SELECT' each, inserting it into a new query (executing one by one). 